# Leash Bio - Predict New Medicines with BELKA

## Introduction
Small molecule drugs play a crucial role in modern medicine, often targeting specific proteins to treat various diseases. However, with a vast chemical space to explore, traditional drug discovery methods can be laborious and time-consuming. The Leash Bio competition, "Predict New Medicines with BELKA," aims to revolutionize small molecule binding prediction by leveraging machine learning techniques.

## Dataset Overview
The competition dataset comprises binary classifications indicating whether a small molecule binds to one of three protein targets. The data were collected using DNA-encoded chemical library (DEL) technology. Each example includes SMILES representations of building blocks and the fully assembled molecule, along with protein target names and binary binding classifications.

### Files
- **train/test.[csv/parquet]:** Contains training or test data in both csv and parquet formats.
  - `id`: Unique identifier for the molecule-binding target pair.
  - `buildingblock1_smiles`, `buildingblock2_smiles`, `buildingblock3_smiles`: SMILES representations of building blocks.
  - `molecule_smiles`: SMILES representation of the fully assembled molecule.
  - `protein_name`: Name of the protein target.
  - `binds`: Binary class label indicating whether the molecule binds to the protein (not available for the test set).
- **sample_submission.csv:** Sample submission file in the correct format.

### Competition Data
Leash Biosciences provides approximately 98M training examples per protein, 200K validation examples per protein, and 360K test molecules per protein. The test set contains building blocks not present in the training set, ensuring generalizability. The datasets are highly imbalanced, with only about 0.5% of examples classified as binders.

## Protein Targets
The competition focuses on predicting binding affinity for three protein targets:

1. **EPHX2 (sEH):** Encoded by the EPHX2 genetic locus, soluble epoxide hydrolase (sEH) is a potential drug target for conditions like high blood pressure and diabetes. Crystal structures and amino acid sequences are provided for contestants wishing to incorporate protein structural information.
2. **BRD4:** Bromodomain 4 plays roles in cancer progression, and inhibiting its activity is a strategy for cancer treatment. Crystal structures and sequences are available for contestants.
3. **ALB (HSA):** Human serum albumin (HSA) is the most common protein in blood and plays a crucial role in drug absorption and transport. Predicting ALB binding can greatly impact drug discovery across various diseases.

<table>
  <tr>
      <th><h2>Protein Name</h2></th>
      <th><h2>Structure</h2></th>
  </tr>
  <tr>
      <td><h2>EPHX2 (sEH)</h2></td>
    <td><img src="https://cdn1.sinobiological.com/styles/default/images/protein-structure/CTSS-protein-structure.jpg" alt="EPHX2 (sEH) protein structure" width="500" height="500"></td>
  </tr>
  <tr>
      <td><h2>BRD4</h2></td>
    <td><img src="https://www.pinclipart.com/picdir/big/70-700834_protein-brd4-pdb-2oss-by-emw-brd4-protein.png" alt="BRD4 protein structure" width="500" height="500"></td>
  </tr>
  <tr>
      <td><h2>ALB (HSA)</h2></td>
    <td><img src="https://cdn.rcsb.org/images/structures/1e78_assembly-1.jpeg" alt="ALB (HSA) protein structure" width="500" height="500"></td>
  </tr>
</table>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leash-BELKA/sample_submission.csv
/kaggle/input/leash-BELKA/train.parquet
/kaggle/input/leash-BELKA/test.parquet
/kaggle/input/leash-BELKA/train.csv
/kaggle/input/leash-BELKA/test.csv


In [2]:
# Install RDKit library for handling chemical data
!pip install rdkit

# Install DuckDB for efficient querying and analysis of large datasets
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 54.4 MB/s eta 0:00:00


In [3]:
# Import necessary libraries
import os  # Operating system utilities
import numpy as np  # Numerical computing library
import pandas as pd  # Data manipulation library
import matplotlib.pyplot as plt  # Data visualization library
import seaborn as sns  # Statistical data visualization library
from tqdm import tqdm  # Progress bar utility
import duckdb  # Embedded SQL database for analytics
import optuna  # Hyperparameter optimization framework
from optuna.samplers import TPESampler  # Tree-structured Parzen Estimator sampler for Optuna
import pickle  # Python object serialization
from rdkit import Chem  # Chemistry informatics library
from rdkit.Chem import AllChem  # Molecular conformer generation
from sklearn.model_selection import train_test_split  # Data splitting utility
from sklearn.preprocessing import OneHotEncoder  # One-hot encoding transformer
from sklearn.ensemble import RandomForestClassifier  # Random forest classifier
from sklearn.ensemble import StackingClassifier  # Stacked ensemble classifier
from xgboost import XGBClassifier  # XGBoost classifier
from catboost import CatBoostClassifier  # CatBoost classifier
from lightgbm import LGBMClassifier  # LightGBM classifier
from sklearn.metrics import average_precision_score  # Average precision score metric

In [4]:
%%time

# Define paths to train and test data
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

# Connect to DuckDB
con = duckdb.connect()

# Query the data from the train.parquet file
df = con.query(f"""(SELECT *
                    FROM parquet_scan('{train_path}')
                    WHERE binds = 0
                    ORDER BY random()
                    LIMIT 30000)
                    UNION ALL
                    (SELECT *
                    FROM parquet_scan('{train_path}')
                    WHERE binds = 1
                    ORDER BY random()
                    LIMIT 30000)""").df()

# Close the DuckDB connection
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2min 23s, sys: 6.23 s, total: 2min 29s
Wall time: 46.5 s


In [5]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,228099182,O=C(Nc1nc2ncc(CNc3ccc(C(=O)O)cc3)nc2c(=O)[nH]1...,Cc1ccc(S(C)(=O)=O)cc1N,N#Cc1cnc2c(C#N)cnn2c1N,Cc1ccc(S(C)(=O)=O)cc1Nc1nc(Nc2nc3ncc(CNc4ccc(C...,sEH,0
1,146055947,O=C(Nc1c(Cl)cc(F)cc1C(=O)O)OCC1c2ccccc2-c2ccccc21,CC1(C)CCOC1CCN,NCc1cccc(C(F)(F)F)c1,CC1(C)CCOC1CCNc1nc(NCc2cccc(C(F)(F)F)c2)nc(Nc2...,sEH,0
2,235911895,O=C(O)C1c2ccccc2CN1C(=O)OCC1c2ccccc2-c2ccccc21,Cl.NCCC1CC1,Cc1nc(CN)sc1C,Cc1nc(CNc2nc(NCCC3CC3)nc(N3Cc4ccccc4C3C(=O)N[D...,HSA,0
3,81926512,O=C(NCC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21,COC1(OC)CC(CN)C1,NCc1cccc(N2CCOCC2)c1,COC1(OC)CC(CNc2nc(NCc3cccc(N4CCOCC4)c3)nc(NCC3...,HSA,0
4,234229276,O=C(O)C1CCN(C(=O)OCC2c3ccccc3-c3ccccc32)C1,Nc1ncncc1Br,Cl.NCCn1cnnn1,O=C(N[Dy])C1CCN(c2nc(NCCn3cnnn3)nc(Nc3ncncc3Br...,HSA,0


In [6]:
%%time
# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

CPU times: user 1min 27s, sys: 1.2 s, total: 1min 28s
Wall time: 1min 28s


In [7]:
%%time

# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

CPU times: user 1.22 s, sys: 222 ms, total: 1.44 s
Wall time: 1.44 s


In [8]:
# Define a list of base models
base_models = [
    ("Random Forest", RandomForestClassifier(random_state=27)),
    ("XGBoost", XGBClassifier(random_state=27)),
    ("CatBoost", CatBoostClassifier(iterations=500, random_state=27))
]

# Train the base models
for model_name, model in tqdm(base_models, desc="Training Base Models"):
    model.fit(X_train, y_train)

# Define the estimators for the Stacked Model
estimators = [(model_name, model) for model_name, model in base_models]

# Create and train the Stacked Model
stacked_model = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier(random_state=27))
stacked_model.fit(X_train, y_train)

# Make predictions on the test set for each model
y_pred_proba_stacked = stacked_model.predict_proba(X_test)[:, 1]

# Calculate the mean average precision for the Stacked Model
map_score_stacked = average_precision_score(y_test, y_pred_proba_stacked)

# Print the mean average precision score for the Stacked Model
print(f"Stacked Model mAP: {map_score_stacked:.8f}")

Training Base Models:  67%|██████▋   | 2/3 [00:58<00:29, 29.91s/it]

Learning rate set to 0.101594
0:	learn: 0.6559903	total: 126ms	remaining: 1m 2s
1:	learn: 0.6266944	total: 176ms	remaining: 43.7s
2:	learn: 0.6002776	total: 216ms	remaining: 35.8s
3:	learn: 0.5808443	total: 258ms	remaining: 31.9s
4:	learn: 0.5669748	total: 297ms	remaining: 29.4s
5:	learn: 0.5532614	total: 339ms	remaining: 27.9s
6:	learn: 0.5417879	total: 383ms	remaining: 27s
7:	learn: 0.5317795	total: 426ms	remaining: 26.2s
8:	learn: 0.5229044	total: 464ms	remaining: 25.3s
9:	learn: 0.5168840	total: 506ms	remaining: 24.8s
10:	learn: 0.5103498	total: 545ms	remaining: 24.2s
11:	learn: 0.5055980	total: 584ms	remaining: 23.8s
12:	learn: 0.4988349	total: 624ms	remaining: 23.4s
13:	learn: 0.4930316	total: 661ms	remaining: 22.9s
14:	learn: 0.4891395	total: 697ms	remaining: 22.5s
15:	learn: 0.4850049	total: 736ms	remaining: 22.3s
16:	learn: 0.4805493	total: 775ms	remaining: 22s
17:	learn: 0.4773061	total: 816ms	remaining: 21.9s
18:	learn: 0.4726554	total: 859ms	remaining: 21.7s
19:	learn: 0.46

Training Base Models: 100%|██████████| 3/3 [01:43<00:00, 34.60s/it]


Learning rate set to 0.101594
0:	learn: 0.6559903	total: 47.5ms	remaining: 23.7s
1:	learn: 0.6266944	total: 92.5ms	remaining: 23s
2:	learn: 0.6002776	total: 133ms	remaining: 22.1s
3:	learn: 0.5808443	total: 188ms	remaining: 23.3s
4:	learn: 0.5669748	total: 228ms	remaining: 22.6s
5:	learn: 0.5532614	total: 271ms	remaining: 22.3s
6:	learn: 0.5417879	total: 317ms	remaining: 22.3s
7:	learn: 0.5317795	total: 362ms	remaining: 22.3s
8:	learn: 0.5229044	total: 403ms	remaining: 22s
9:	learn: 0.5168840	total: 445ms	remaining: 21.8s
10:	learn: 0.5103498	total: 487ms	remaining: 21.6s
11:	learn: 0.5055980	total: 524ms	remaining: 21.3s
12:	learn: 0.4988349	total: 564ms	remaining: 21.1s
13:	learn: 0.4930316	total: 599ms	remaining: 20.8s
14:	learn: 0.4891395	total: 637ms	remaining: 20.6s
15:	learn: 0.4850049	total: 678ms	remaining: 20.5s
16:	learn: 0.4805493	total: 719ms	remaining: 20.4s
17:	learn: 0.4773061	total: 759ms	remaining: 20.3s
18:	learn: 0.4726554	total: 801ms	remaining: 20.3s
19:	learn: 0.

In [9]:
%%time

# Process the test.csv file chunk by chunk
test_file = '/kaggle/input/leash-BELKA/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.csv file into a pandas DataFrame
# Wrap the loop with tqdm to display progress
for df_test in tqdm(pd.read_csv(test_file, chunksize=10_000)):
    
    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities using the best model (Stacked Model)
    probabilities = stacked_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))

168it [1:07:31, 24.12s/it]

CPU times: user 1h 7min 35s, sys: 36.8 s, total: 1h 8min 12s
Wall time: 1h 7min 31s
